In [ ]:
import datetime

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly
from dash.dependencies import Input, Output
import random
import http.client
import json
from skimage.color import rgb2hsv

external_stylesheets = ['https://github.com/Owaiskhan9654/DigiGene/blob/bdf66f620f8b75b59f7a4e4687508ec908b9a6c8/DigiGene.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    html.Div([
        html.Div(id='live-update-text',style = {'padding': '5px', 'fontSize': '32px','textAlign': 'center',\
                                              'backgroundColor':'rgb(17, 17, 17)','color':'white'}),
        dcc.Graph(id='live-update-graph'),
        dcc.Interval(
            id='interval-component',
            interval=1*5000, # in milliseconds
            n_intervals=0
        )
    ],),
)


def rgb_to_hex(r, g, b):
    return ('{:x}{:x}{:x}').format(r, g, b)



@app.callback(Output('live-update-text', 'children'),
              Input('interval-component', 'n_intervals'))
def update_metrics(n):
    #Current_color = open("Current_color.txt","r")
    #Current_color= Current_color.readlines()
    
    conn = http.client.HTTPSConnection("getpantry.cloud")
    payload = ''
    headers = {
      'Content-Type': 'application/json'
    }
    conn.request("GET", "/apiv1/pantry/2cb1189d-f1e5-4acb-8fba-c86b8282f25f/basket/DigiGeneV1", payload, headers)
    res = conn.getresponse()
    data_pantry = res.read()

    data_pantry = json.loads(data_pantry.decode('utf-8'))
    Current_color = data_pantry["current"]
    
    def int_custom(a):
        try:
            return(int(a))
        except :
            return 0

    Current_color = Current_color.split(' ,')
    Current_color = [ int_custom(i) for i in Current_color]
    
    RedSensor1 = Current_color[0]
    GreenSensor1 = Current_color[1]
    BlueSensor1 = Current_color[2]
    RedSensor2 = Current_color[4]
    GreenSensor2 = Current_color[5]
    BlueSensor2 = Current_color[6]
    RedSensor3 = Current_color[8]
    GreenSensor3 = Current_color[9]
    BlueSensor3 = Current_color[10]
    
    #print('RedSensor1 ',Current_color[0],'GreenSensor1 ',Current_color[1],'BlueSensor1 ', Current_color[2])
    Sensor1_name = '#' + rgb_to_hex(RedSensor1, GreenSensor1, BlueSensor1)
    Sensor2_name = '#' + rgb_to_hex(RedSensor2, GreenSensor2, BlueSensor2)
    Sensor3_name = '#' + rgb_to_hex(RedSensor3, GreenSensor3, BlueSensor3)
    style = {'padding': '5px', 'fontSize': '32px','textAlign': 'center','backgroundColor':'rgb(17, 17, 17)','color':'white'}
    return [
        
        html.Span('DigiGene Live Feed', style=style),\
        html.Br(),
        html.Span('Sensor 1:  Red {0:.2f}, Sensor 1: Green {1:.2f}, Sensor 1: Blue {2:.2f}, Green1/Blue1: {3:.2f},\
        Blue1/Red1: {4:.2f}'.format(RedSensor1,GreenSensor1,BlueSensor1,GreenSensor1/BlueSensor1,BlueSensor1/RedSensor1,)\
                  , style=style),\
        html.Br(),
        html.Span('Sensor 2: Red {0:.2f} , Sensor 2: Green {1:.2f}, Sensor 2: Blue {2:.2f},Green2/Blue2: {3:.2f},Blue2/Red2: {4:.2f}\
        '.format(RedSensor2,GreenSensor2,BlueSensor2,GreenSensor2/BlueSensor2,BlueSensor2/RedSensor2,), style=style)
        ,html.Br(),
        html.Span('Sensor 3: Red {0:0.2f} , Sensor 3: Green {1:.2f}, Sensor 3: Blue {2:.2f}, Green3/Blue3: {3:.2f},Blue3/Red3: {4:.2f}\
        '.format(RedSensor3,GreenSensor3,BlueSensor3,GreenSensor3/BlueSensor3,BlueSensor3/RedSensor3,), style=style),
        html.Br(),
        html.Div(className='row',
         style = {'display' : 'flex','color':'white','padding-left': '300px','padding-right': '300px'},
                 children=[ 
                     html.Div('Sensor 1 Color',style={'color':'white','backgroundColor':'rgb(17, 17, 17)','padding': '35px', 'fontSize': '32px',\
                                                      'textAlign': 'center', },className='col s6 m6',),\
                     html.Div('                 ',style={'backgroundColor':Sensor1_name,'padding': '35px', 'fontSize': '32px',\
                                                         'textAlign': 'center','borderRadius': '15px', },className='col s6 m6',),\
             
                     html.Div('Sensor 2 Color',style={'color':'white','backgroundColor':'rgb(17, 17, 17)','padding': '35px', 'fontSize': '32px',\
                                                         'textAlign': 'center', },className='col s6 m6',),\
                     html.Div('               ',style={'backgroundColor':Sensor2_name,'padding': '35px', 'fontSize': '32px',\
                                                          'textAlign': 'center','borderRadius': '15px', },className='col s6 m6',),\

                     html.Div('Sensor 3 Color',style={'color':'white','backgroundColor':'rgb(17, 17, 17)','padding': '35px', 'fontSize': '32px',\
                                                          'textAlign': 'center', },className='col s6 m6',),\


                     html.Div('               ',style={'backgroundColor':Sensor3_name,'padding': '35px', 'fontSize': '32px',\
                                                     'textAlign': 'center','borderRadius': '15px', },className='col s6 m6',)])
    ]
#'+str(int(RedSensor1))+','+str(int(GreenSensor1))+ ','+str(int(BlueSensor1))+'

# Multiple components can update everytime interval gets fired.
@app.callback(Output('live-update-graph', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_graph_live(n):

    data = {
        'time': [],
        'Red Sensor 1': [],
        'Green Sensor 1': [],
        'Blue Sensor 1': [],
        'Red Sensor 2': [],
        'Green Sensor 2': [],
        'Blue Sensor 2': [],
        'Red Sensor 3': [],
        'Green Sensor 3': [],
        'Blue Sensor 3': [],
    }
    conn = http.client.HTTPSConnection("getpantry.cloud")
    payload = ''
    headers = {
          'Content-Type': 'application/json'
        }
    # Collect some data
    for i in range(6):
        time = datetime.datetime.now() - datetime.timedelta(seconds=i*3)
        #Current_color = open("Current_color.txt","r")
        #Current_color= Current_color.readlines()
        #print(Current_color)
        

        conn.request("GET", "/apiv1/pantry/2cb1189d-f1e5-4acb-8fba-c86b8282f25f/basket/DigiGeneV1", payload, headers)
        res = conn.getresponse()
        data_pantry = res.read()

        data_pantry = json.loads(data_pantry.decode('utf-8'))
        Current_color = data_pantry["current"]
        
        
        
        def int_custom(a):
            try:
                return(int(a))
            except :
                return 0

        Current_color = Current_color.split(' ,')
        Current_color = [ int_custom(i) for i in Current_color]

        RedSensor1 = Current_color[0]
        GreenSensor1 = Current_color[1]
        BlueSensor1 = Current_color[2]
        RedSensor2 = Current_color[4]
        GreenSensor2 = Current_color[5]
        BlueSensor2 = Current_color[6]
        RedSensor3 = Current_color[8]
        GreenSensor3 = Current_color[9]
        BlueSensor3 = Current_color[10]
        
        #RedSensor1 = random.randint(1, 25)
        #GreenSensor1 = random.randint(1, 55)
        #BlueSensor1 = random.randint(20, 25)
        #RedSensor2 = random.randint(50, 75)
        #GreenSensor2 = random.randint(60, 95)
        #BlueSensor2 = random.randint(45, 55)
        #RedSensor3 = random.randint(1, 25)
        #GreenSensor3 = random.randint(1, 55)
        #BlueSensor3 = random.randint(1, 35)
        data['Red Sensor 1'].append(RedSensor1)
        data['Green Sensor 1'].append(GreenSensor1)
        data['Blue Sensor 1'].append(BlueSensor1)
        
        data['Red Sensor 2'].append(RedSensor2)
        data['Green Sensor 2'].append(GreenSensor2)
        data['Blue Sensor 2'].append(BlueSensor2)
        
        data['Red Sensor 3'].append(RedSensor3)
        data['Green Sensor 3'].append(GreenSensor3)
        data['Blue Sensor 3'].append(BlueSensor3)
        data['time'].append(time)

    # Create the graph with subplots
    fig = plotly.tools.make_subplots(rows=3, cols=3, vertical_spacing=0.2,subplot_titles=[
        'Red Sensor 1', 
        'Green Sensor 1', 
        'Blue Sensor 1', 
        'Red Sensor 2', 
        'Green Sensor 2', 
        'Blue Sensor 2',
        'Red Sensor 3', 
        'Green Sensor 3', 
        'Blue Sensor 3',
    ],)
    fig['layout']['margin'] = {
        'l': 50, 'r': 10, 'b': 10, 't': 60
    }
    #fig['layout']['legend'] = {'x': 0, 'y': 1, 'xanchor': 'left'}

    #fig.add_bar(
     #   x =data['time'],
     #   y= data['Red Sensor 1'],
     #   name= 'Red Sensor 1',
        #width=800,
        
        #'line':dict(width=1, color='red')
     #   marker=dict(color="Red"),
     #   row=1, col=1)
    
    fig.append_trace({
        'x': data['time'],
        'y': data['Red Sensor 1'],
        'name': 'Red Sensor 1',
        'mode':'lines',
        'line':dict(width=1, color='Red')
    }, 1, 1)
    
    fig.append_trace({
        'x': data['time'],
        'y': data['Green Sensor 1'],
        'name': 'Green Sensor 1',
        'mode':'lines',
        'line':dict(width=1, color='green')
    }, 1, 2)
    
    fig.append_trace({
        'x': data['time'],
        'y': data['Blue Sensor 1'],
        'name': 'Blue Sensor 1',
        'mode':  'lines',          #'lines+markers',
        'line':dict(width=1, color='blue')
    }, 1, 3)
    
    
    fig.append_trace({
        'x': data['time'],
        'y': data['Red Sensor 2'],
        'name': 'Red Sensor 2',
        'mode':'lines',
        
        'line':dict(width=1, color='red')
    }, 2, 1)
    
    fig.append_trace({
        'x': data['time'],
        'y': data['Green Sensor 2'],
        'name': 'Green Sensor 2',
        'mode':'lines',
        'line':dict(width=1, color='green')
    }, 2, 2)
    
    fig.append_trace({
        'x': data['time'],
        'y': data['Blue Sensor 2'],
        'name': 'Blue Sensor 2',
        'mode':'lines',
        'line':dict(width=1, color='blue')
    }, 2, 3)
    
    
    fig.append_trace({
        'x': data['time'],
        'y': data['Red Sensor 3'],
        'name': 'Red Sensor 3',
        'mode':'lines',
        
        'line':dict(width=1, color='red')
    }, 3, 1)
    
    fig.append_trace({
        'x': data['time'],
        'y': data['Green Sensor 3'],
        'name': 'Green Sensor 3',
        'mode':'lines',
        'line':dict(width=1, color='green')
    }, 3, 2)
    
    fig.append_trace({
        'x': data['time'],
        'y': data['Blue Sensor 3'],
        'name': 'Blue Sensor 3',
        'mode':'lines',
        'line':dict(width=1, color='blue')
    }, 3, 3)

    fig.update_yaxes(range=[0, 255], dtick=30)
    fig.update_layout(height=1200, width=1900,template='plotly_dark')
    return fig


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/



C:\Users\Admin\AppData\Local\Temp/ipykernel_16568/3609016769.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Admin\AppData\Local\Temp/ipykernel_16568/3609016769.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jan/2022 19:18:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:18:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:18:06] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:18:06] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:18:06] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:18:07] "POST /_dash-update-component HTTP/1.1" 200 -
c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\plotly\tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

127.0.0.1 - - [05/Jan/2022 19:18:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:18:12] "POST /_dash-update-component HTTP/1.1" 200 -
c:\users\admin\appdata\local\programs\python\python38\lib\si

127.0.0.1 - - [05/Jan/2022 19:21:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:21:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:21:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:21:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:21:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:22:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:22:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:22:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:22:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:22:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:22:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 19:22:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.

In [ ]:
'''Current_color = open("Current_color.txt","r")
Current_color= Current_color.readlines()
print(Current_color)
def int_custom(a):
    try:
        return(int(a))
    except :
        return 0

Current_color = [ i.replace("\n","") for i in Current_color]
Current_color = Current_color[0].split(' ,')
Current_color = [ int_custom(i) for i in Current_color]
Current_color'''

In [ ]:
Current_color[0]

In [ ]:
import http.client
import json

conn = http.client.HTTPSConnection("getpantry.cloud")
payload = ''
headers = {
  'Content-Type': 'application/json'
}
conn.request("GET", "/apiv1/pantry/2cb1189d-f1e5-4acb-8fba-c86b8282f25f/basket/DigiGeneV1/", payload, headers)
res = conn.getresponse()
data = res.read()
data = json.loads(data.decode('utf-8'))
Current_color = data["current"]
print(type(Current_color))
def int_custom(a):
    try:
        return(int(a))
    except :
        return 0

#Current_color = [ i.replace("\n","") for i in Current_color]
Current_color = Current_color.split(' ,')
Current_color = [ int_custom(i) for i in Current_color]
Current_color[0]

In [ ]:
'''import http.client
import json

conn = http.client.HTTPSConnection("getpantry.cloud")
payload = json.dumps({
  "newKey1234": "newValue1234"
})
headers = {
  'Content-Type': 'application/json'
}
conn.request("PUT", "/apiv1/pantry/33b4291b-ac00-4cf2-aa99-806b22988f3c/basket/DigiGene", payload, headers)
res = conn.getresponse()
data = res.read().decode("utf-8")
print(data)'''

In [ ]:
https://getpantry.cloud/apiv1/pantry/33b4291b-ac00-4cf2-aa99-806b22988f3c/basket/DigiGene

In [4]:
'#'+rgb_to_hex(31, 75, 88)

'#1f4b58'

In [3]:
def rgb_to_hex(r, g, b):
    return ('{:x}{:x}{:x}').format(r, g, b)